In [36]:
#ابوذر بختیاری برزیده

import scipy.io
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy.signal import find_peaks
from scipy.stats import kurtosis
from sklearn.model_selection import train_test_split


In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install --upgrade --no-cache-dir gdown
!gdown 15CfPVJJlfJajLqXYvIf1ObkqrLiWAp45


Mounted at /content/drive
  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
/usr/local/lib/python3.10/dist-packages/gdown/download.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(line, features="html.parser")
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=15CfPVJJlfJajLqXYvIf1ObkqrLiWAp45

but Gdown can't. Please check connections and permissions.


In [6]:
data99 = scipy.io.loadmat('/content/drive/MyDrive/machine-learning/mini project 1/2-1/99.mat')
data106 = scipy.io.loadmat('/content/drive/MyDrive/machine-learning/mini project 1/2-1/106.mat')

In [7]:
normal = data99['X099_DE_time']
fault1 = data106['X106_DE_time']
print(f'number of data in X106_DE_time(fault) :{fault1.size}')
print(f'number of data in X099_DE_time(normal) :{normal.size}')




number of data in X106_DE_time(fault) :121991
number of data in X099_DE_time(normal) :485063


In [8]:
print(normal.shape)
print(normal)

(485063, 1)
[[ 0.06425354]
 [ 0.06300185]
 [-0.00438092]
 ...
 [ 0.00521538]
 [-0.06571385]
 [-0.12266585]]


In [9]:
N = 275
M = 300
C1 = np.array([normal[N * i:N * (i + 1)] for i in range(M)]).reshape(N, M)
C2 = np.array([fault1[N * i:N * (i + 1)] for i in range(M)]).reshape(N, M)
print(C1.shape)
print(C2.shape)

(275, 300)
(275, 300)


In [10]:
X = np.vstack((C1, C2))
X.shape

(550, 300)

#2_b

In [12]:
data_std = np.std(X, axis=1, keepdims=True)
data_rms = np.sqrt(np.mean(np.square(X), axis=1)).reshape(-1, 1)
data_peak=np.max(np.abs(X), axis=1).reshape(-1, 1)
data_abs_mean=np.mean(np.abs(X), axis=1).reshape(-1, 1)
data_impact_factor = np.max(np.abs(X), axis=1).reshape(-1, 1) / np.mean(np.abs(X), axis=1).reshape(-1, 1)
data_smr= np.square(np.mean(np.sqrt(np.abs(X)), axis=1)).reshape(-1, 1)
data_skewness= np.mean(((X - np.mean(X, axis=1, keepdims=True)) ** 3) / (np.std(X, axis=1, keepdims=True) ** 3), axis=1).reshape(-1, 1)
data_cf=np.max(np.abs(X), axis=1).reshape(-1, 1) / np.sqrt(np.mean(np.square(X), axis=1)).reshape(-1, 1)


X_new = np.hstack([data_std, data_rms,data_peak,data_abs_mean,data_impact_factor,data_smr,data_skewness,data_cf])
X_new.shape

(550, 8)

In [14]:
def data_split(X, per):
    limit = int(len(X) * per)
    idx = np.random.permutation(len(X))
    return idx[:limit], idx[limit:]

d_train = 0.8
i_train1, i_test1 = data_split(X_new[:N], d_train)
i_train2, i_test2 = data_split(X_new[N:], d_train)
i_test1

array([204, 241, 237, 100, 173, 202, 198, 158,  23, 190, 211,   7, 221,
       201, 251, 266,  69,   8, 217, 126, 168, 104,  92, 197,  11, 263,
       210,  83, 213,  15,  26, 188, 231, 257, 271, 119,   0,  51, 128,
       162,  89, 116, 178, 108,  81, 154, 156, 142, 160, 151,  70, 258,
       106, 206, 192])

In [15]:
C1_train, C1_test = X_new[:N][i_train1], X_new[:N][i_test1]
C2_train, C2_test = X_new[N:][i_train2], X_new[N:][i_test2]

X_train = np.vstack((C1_train, C2_train))
y_train = np.vstack((np.ones((len(C1_train), 1)), 2 * np.ones((len(C2_train), 1))))
X_test = np.vstack((C1_test, C2_test))
y_test = np.vstack((np.ones((len(C1_test), 1)), 2 * np.ones((len(C2_test), 1))))

print('data_1 X_train: ', X_train.shape)
print('data 1 y_train: ', y_train.shape)
print('data_2 X_test: ', X_test.shape)
print('data_2 y_test: ', y_test.shape)

data_1 X_train:  (440, 8)
data 1 y_train:  (440, 1)
data_2 X_test:  (110, 8)
data_2 y_test:  (110, 1)


In [16]:
def Preprocessing(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma


X_train = Preprocessing(X_train)
X_test = Preprocessing(X_test)

print(X_train.shape)
print(X_test.shape)


(440, 8)
(110, 8)


In [25]:
train_df = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1))), columns=[f'Feature {i+1}' for i in range(X_train.shape[1])] + ['Label'])
test_df = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1))), columns=[f'Feature {i+1}' for i in range(X_test.shape[1])] + ['Label'])

train_df.to_csv('/content/drive/MyDrive/machine-learning/mini project 1/2-1/train_data.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/machine-learning/mini project 1/2-1/test_data.csv', index=False)


In [26]:
print(test_df.shape)
print(test_df.columns)

(110, 9)
Index(['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5',
       'Feature 6', 'Feature 7', 'Feature 8', 'Label'],
      dtype='object')


In [29]:
df_test = pd.read_csv('/content/drive/MyDrive/machine-learning/mini project 1/2-1/test_data.csv')
df_train = pd.read_csv('/content/drive/MyDrive/machine-learning/mini project 1/2-1/train_data.csv')

In [32]:
df_test, df_train

(     data_std  data_rms  data_skewness  data_peak  data_crest_factor  \
 0   -1.025978 -1.029674      -1.049263  -1.027836          -1.327488   
 1   -1.020124 -1.017617      -1.010330  -1.012379          -1.076059   
 2   -0.960396 -0.953579      -0.964074  -0.945366          -0.970356   
 3   -0.927825 -0.923558      -0.885439  -0.936504          -0.456529   
 4   -1.047793 -1.044694      -0.989900  -1.052611          -0.753376   
 ..        ...       ...            ...        ...                ...   
 105  1.140576  1.141065       1.036517   1.129965           0.798097   
 106  1.051762  1.051911       1.031115   1.050452           0.911125   
 107  0.979466  0.979250       1.035617   0.910155           1.154822   
 108  1.022011  1.021796       0.944375   1.044927           0.760348   
 109  1.013423  1.013794       0.833623   1.092389           0.488514   
 
      data_absolute_mean  data_impact_factor  data_square_mean_root  label  
 0             -1.029583           -0.502789 

In [33]:
df_test.columns = ['data_std', 'data_rms', 'data_skewness', 'data_peak', 'data_crest_factor', 'data_absolute_mean','data_impact_factor','data_square_mean_root', 'label']
df_train.columns = ['data_std', 'data_rms', 'data_skewness', 'data_peak', 'data_crest_factor', 'data_absolute_mean','data_impact_factor','data_square_mean_root', 'label']

df_test, df_train

(     data_std  data_rms  data_skewness  data_peak  data_crest_factor  \
 0   -1.025978 -1.029674      -1.049263  -1.027836          -1.327488   
 1   -1.020124 -1.017617      -1.010330  -1.012379          -1.076059   
 2   -0.960396 -0.953579      -0.964074  -0.945366          -0.970356   
 3   -0.927825 -0.923558      -0.885439  -0.936504          -0.456529   
 4   -1.047793 -1.044694      -0.989900  -1.052611          -0.753376   
 ..        ...       ...            ...        ...                ...   
 105  1.140576  1.141065       1.036517   1.129965           0.798097   
 106  1.051762  1.051911       1.031115   1.050452           0.911125   
 107  0.979466  0.979250       1.035617   0.910155           1.154822   
 108  1.022011  1.021796       0.944375   1.044927           0.760348   
 109  1.013423  1.013794       0.833623   1.092389           0.488514   
 
      data_absolute_mean  data_impact_factor  data_square_mean_root  label  
 0             -1.029583           -0.502789 

#**2-3**

In [34]:
x_train = df_train[['data_std', 'data_rms', 'data_skewness', 'data_peak', 'data_crest_factor', 'data_absolute_mean','data_impact_factor','data_square_mean_root']].values
y_train = df_train[['label']].values
x_test= df_test[['data_std', 'data_rms', 'data_skewness', 'data_peak', 'data_crest_factor', 'data_absolute_mean','data_impact_factor','data_square_mean_root']].values
y_test=df_test[['label']].values
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(440, 8)
(440, 1)
(110, 8)
(110, 1)


In [37]:
X_combined = np.vstack((x_train, x_test))
y_combined = np.vstack((y_train, y_test))

print(f'all number data :{X_combined.shape}')

X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined, y_combined, test_size=0.2, random_state=4,shuffle=True)

print(X_train_combined.shape)
print(y_train_combined.shape)
print(X_test_combined.shape)
print(y_test_combined.shape)


all number data :(550, 8)
(440, 8)
(440, 1)
(110, 8)
(110, 1)


#2-4

In [40]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train_combined)

X_test_scaled = scaler.transform(X_test_combined)
X_train_scaled

array([[0.9268281 , 0.92647269, 0.74770877, ..., 0.94549034, 0.64898733,
        0.49724141],
       [0.03752549, 0.04275431, 0.02545544, ..., 0.05611712, 0.27864763,
        0.15297233],
       [0.07199871, 0.07542169, 0.09189845, ..., 0.09667823, 0.        ,
        0.46477921],
       ...,
       [0.81736991, 0.81656711, 0.85319405, ..., 0.75613784, 0.8812877 ,
        0.79632544],
       [0.04160791, 0.04151545, 0.02847558, ..., 0.05024047, 0.41925232,
        0.18045261],
       [0.05895031, 0.05587228, 0.05306812, ..., 0.07169172, 0.7087954 ,
        0.29564864]])

In [41]:
from sklearn.preprocessing import StandardScaler

Standard_scaler = StandardScaler()

X_train_Standardscaler = Standard_scaler.fit_transform(X_train_combined)

X_test_Standardscaler = Standard_scaler.transform(X_test_combined)
X_train_Standardscaler

array([[ 1.20373693,  1.20460243,  0.97414882, ...,  1.30846264,
         0.4321888 ,  0.45811652],
       [-1.03718626, -1.0230429 , -1.00225776, ..., -1.02447505,
        -1.77559198, -0.93790875],
       [-0.95031837, -0.94069613, -0.8204401 , ..., -0.91807816,
        -3.43675035,  0.32648097],
       ...,
       [ 0.92791699,  0.92755641,  1.2628035 , ...,  0.81176724,
         1.81704782,  1.67091456],
       [-1.02689911, -1.02616577, -0.99399332, ..., -1.03989023,
        -0.9373769 , -0.82647509],
       [-0.98319859, -0.98997559, -0.92669718, ..., -0.98362091,
         0.78873468, -0.35935047]])

In [43]:
print(df_train.max())
print()
print(df_train.min())

data_std                 1.381354
data_rms                 1.383110
data_skewness            1.668815
data_peak                1.474924
data_crest_factor        2.436574
data_absolute_mean       1.539886
data_impact_factor       2.543017
data_square_mean_root    2.522856
label                    2.000000
dtype: float64

data_std                -1.135715
data_rms                -1.134556
data_skewness           -1.076628
data_peak               -1.159320
data_crest_factor       -1.491256
data_absolute_mean      -1.174171
data_impact_factor      -3.468511
data_square_mean_root   -1.577290
label                    1.000000
dtype: float64
